In [17]:
# Import Dependencies
import pandas as pd
import re
import xml.etree.ElementTree as ET
import os

### **Step 1:** Change the Input File Header Names

1. Rename the header names in the input file to match the corresponding tag names in the XML schema.

2. If a value and unit are needed, rename the header name followed by a "`/`", for example, `TensileModulus/unit`, `Other_Processing/ChooseParameter/Drying-Evaporation/Temperature/value`.

3. Update the path to the location of your input files and specify the destination where you want to save your output XML files.

In [18]:
# Define the path to the input xml schema file
input_schema_file = r'C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/XML-Schema.xml'

# Define the path to the input file
input_file = r'C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/resources/input_file_Boran_data/Boran_full_set_updated-v2.xlsx'

# Define the directory for the output xml files
output_dir = r'C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/resources/output_file_Boran_data/'

### **Step 2:** Convert the input file into a XML file

Note: The tags "`SIMULATION-FEA`" and "`MATERIALPROPERTIES-FEA`" are skipped in this code.

In [19]:
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load Excel data
df = pd.read_excel(input_file)

In [20]:
def add_authors_to_element(element, row):
    """
    Add each author as a new <Author> element under the specified parent element.
    
    Args:
    element (xml.etree.ElementTree.Element): The parent XML element where the <Author> elements should be added.
    row (pd.Series): A single row of the dataframe containing the data to be added.
    """
    # Use a set to keep track of added authors to prevent duplicates
    added_authors = set()
    
    # Filter out all columns that start with 'Author'
    author_columns = [col for col in row.index if 'Author' in col]
    
    for col in author_columns:
        author_name = str(row[col]).strip()
        if pd.notna(author_name) and author_name not in added_authors:
            # Directly add each author to the parent element without nesting
            author_element = ET.SubElement(element, "Author")
            author_element.text = author_name
            added_authors.add(author_name)  # Add the author to the set to prevent duplicates

def add_data_to_element(element, row, parent_path_parts=None):
    """
    Recursively add content to XML elements based on provided row data and XML schema.
    
    Args:
    element (xml.etree.ElementTree.Element): The XML element to which data should be added.
    row (pd.Series): A single row of the dataframe containing the data to be added.
    parent_path_parts (list of str): The list of path parts of the XML tag, used to match with Excel headers.
    """
    if parent_path_parts is None:
        parent_path_parts = []

    # Define elements to be skipped
    skip_elements = {"SIMULATION-FEA", "MATERIALPROPERTIES-FEA"}

    for child in list(element):
        # Check if the current element should be skipped
        if child.tag in skip_elements:
            print(f"Skipping element: {child.tag}")
            continue  # Skip processing this element

        # Update the path parts list
        current_path_parts = parent_path_parts + [child.tag]

        # Special handling for "Author"
        if child.tag == "Author":
            add_authors_to_element(element, row)
            continue  # Stop further recursion for 'Author' to prevent nesting

        # Process non-author elements
        elif len(list(child)) == 0:  # Only handle elements without child elements
            for col in row.index:
                col_parts = col.split('/')
                if col_parts == current_path_parts[-len(col_parts):]:
                    value = row[col]
                    if pd.notna(value):
                        child.text = str(value)

        # Recursively process child elements
        add_data_to_element(child, row, current_path_parts)

def remove_empty_elements(element):
    """
    Recursively remove elements that have no data.
    
    Args:
    element (xml.etree.ElementTree.Element): The XML element to be checked for empty children.
    """
    for child in list(element):
        remove_empty_elements(child)
        # Remove the element if it has no text and no children
        if not (child.text and child.text.strip()) and len(child) == 0:
            element.remove(child)

# Process each row in the DataFrame and generate separate XML files
for index, row in df.iterrows():
    # Reload the XML schema for each row to ensure each file starts from the original schema
    tree = ET.parse(input_schema_file)
    root = tree.getroot()

    # Add data to the XML structure based on the schema and Excel headers
    add_data_to_element(root, row)

    # Remove elements that have no data
    remove_empty_elements(root)

    # Get the ID for the current row
    sample_id = row['ID']  # Assuming 'ID' is the column name in your Excel file

    # Define the output file path for each sample using the ID
    output_file = os.path.join(output_dir, f'{sample_id}.xml')

    # Write the modified XML data to a file
    tree.write(output_file, encoding='utf-8', xml_declaration=True)

    print(f'XML file for sample {sample_id} has been created and saved as {output_file}')

Skipping element: SIMULATION-FEA
Skipping element: MATERIALPROPERTIES-FEA
XML file for sample I_A_AC has been created and saved as C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/resources/output_file_Boran_data/I_A_AC.xml
Skipping element: SIMULATION-FEA
Skipping element: MATERIALPROPERTIES-FEA
XML file for sample I_A025 has been created and saved as C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/resources/output_file_Boran_data/I_A025.xml
Skipping element: SIMULATION-FEA
Skipping element: MATERIALPROPERTIES-FEA
XML file for sample I_A050 has been created and saved as C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/resources/output_file_Boran_data/I_A050.xml
Skipping element: SIMULATION-FEA
Skipping element: MATERIALPROPERTIES-FEA
XML file for sample I_A075 has been created and saved as C:/Users/jhyan/Documents/GitHub-Projects/Duke-MaterialsMine-Input-Convert/resources/output_file_Boran_data/I_A075.xml
Skip